#### Stats, Head and Body
The base stats of a fused Pokémon are determined by the base stats of its head and body Pokémon. The formula is a weighted average with a bias towards the **body Pokémon for physical stats** (Atk, Def, Spd) and a bias towards the **head Pokémon for special stats (HP, Sp.Atk, Sp.Def)**.

- ATK, DEF, SPD

fusionStat = 2 * bodyStat / 3 + headStat / 3

- HP, SP.DEF, SP.ATK

fusionStat = 2 * headStat / 3 + bodyStat/3


#### name, Nature, and Moves
You will be able to choose a name, nature, and set of moves based on what either Pokémon has at the time. The Move Reminder found on the top floor of Pokémon centers can teach moves from either original Pokémon.

Nature can be changed in the far southwest of Cinnabar Island.

#### Ability
A fusion has access to any of the abilities of the Fused pokemon. When fusing, You are only able to choose between the two abilities that the Parent Pokemon had at the time of fusion. Ex: fusing Politoed with damp, and Glaceon with snow cloak will only show those two abilities, regardless of fusion order.

#### EVs and IVs
Fusion keeps the EVs of the body and discards the EVs of the head.

If using DNA splicers, an average of both Pokémon's IVs is used. If using super splicers, the higher IV in each stat is used.

#### Typing

The fused Pokémon will have its head's primary type and its body's secondary type. If the body's Pokémon doesn't have a secondary type, its primary type will be used instead.

If the head is already providing the element the body wants to provide, The body will provide its primary type instead. A Grimer/Oddish for example is Poison/Grass. Oddish normally provides Poison as a body, but Grimer already provides Poison; so to avoid redundancy, Oddish instead provides its primary type, Grass.

In [1]:
import numpy as np
import pandas as pd

pokedex = pd.read_csv("data/CompleteInfiniteFusionDex_v6.2.3.csv")

type_chart = pd.read_csv("data/typingchart.csv")

type_chart = type_chart.set_index("Attacking")
type_chart.columns = type_chart.columns.str.lower()
type_chart.index = type_chart.index.str.lower()

In [2]:
def pkm_query(pokemon:str,form:str="base"):
    return pokedex.loc[(pokedex.name == pokemon.lower())&(pokedex.form == form)]

In [3]:
def weighted_avg(stat1,stat2):
    return int((2*stat1+stat2)/3)

In [4]:
def calc_fusion_stats(fusion_head, fusion_body):
    fusion_stats = {}
    # Pandas doc recommends using to_numpy() instead of values
    fusion_stats["hp"] = weighted_avg(fusion_head["hp"].values[0],fusion_body["hp"].values[0])
    fusion_stats["atk"] = weighted_avg(fusion_body["attack"].values[0],fusion_head["attack"].values[0])
    fusion_stats["def"] = weighted_avg(fusion_body["defense"].values[0],fusion_head["defense"].values[0])
    fusion_stats["spatk"] = weighted_avg(fusion_head["sp_attack"].values[0],fusion_body["sp_attack"].values[0])
    fusion_stats["spdef"] = weighted_avg(fusion_head["sp_defense"].values[0],fusion_body["sp_defense"].values[0])
    fusion_stats["spe"] = weighted_avg(fusion_body["speed"].values[0],fusion_head["speed"].values[0])
    fusion_stats["total"] = sum(fusion_stats.values())
    return fusion_stats

In [5]:
def calc_fusion_typing(fusion_head,fusion_body):
    fusion_types = {}
    fusion_types["type_1"] = fusion_head["type_1"].values[0]
    if fusion_body["type_2"].values[0] == fusion_head["type_1"].values[0]:
        fusion_types["type_2"] = fusion_body["type_1"].values[0]
    else:
        fusion_types["type_2"] = fusion_body["type_2"].values[0]
    return fusion_types

In [6]:
def calc_fusion_weakness(fusion_typing:dict):
    if fusion_typing['type_1'] == fusion_typing['type_2']:
        pkm_weak = type_chart[fusion_typing['type_1']]
    else:
        pkm_weak = type_chart[fusion_typing['type_1']]*type_chart[fusion_typing['type_2']]

    fusion_weak = {}
    for value in pkm_weak:
        if value not in fusion_weak:
            fusion_weak[str(value)] = pkm_weak[pkm_weak == value].index.tolist()

    return fusion_weak

In [7]:
from dataclasses import dataclass
@dataclass
class Fusion:
    head:str
    body:str
    types:dict
    stats:dict
    
    def __init__(self,head,body,forms):
        self.head = pkm_query(head,forms[0])
        self.body = pkm_query(body,forms[1])
        
        self.types = calc_fusion_typing(self.head,self.body)
        self.stats = calc_fusion_stats(self.head,self.body)
        
    def print_weakness(self):
        weak = calc_fusion_weakness(self.types)
        keys = list(weak.keys())
        keys.sort(reverse=True)
        for key in keys:
            print(f"{key} ({len(weak[key])}):{weak[key]}")
    
    def possible_abilities(self):
        abilites = {}
        abilites['regular'] = [self.head.ability_1.values[0], self.head.ability_2.values[0], self.body.ability_1.values[0], self.body.ability_2.values[0]]
        abilites['hidden'] = [self.head.ability_hidden.values[0], self.body.ability_hidden.values[0]]
        for key in abilites.keys():
            abilites[key] = [x for x in abilites[key] if x != 'none']
        return abilites

In [8]:
def calc_fusion(pokemon1:str, pokemon2:str,forms:list):
    return Fusion(pokemon1,pokemon2,forms)

## Fusion Calculator

In [9]:
def fusion_calculator(pokemon1:str,pokemon2:str,pkm1_form:str="base",pkm2_form:str="base"):
    fusions = [calc_fusion(pokemon1, pokemon2,[pkm1_form,pkm2_form]),calc_fusion(pokemon2, pokemon1,[pkm2_form,pkm1_form])]
    stats_diff = {}
    for key in fusions[0].stats.keys():
        stats_diff[key] = fusions[0].stats[key] - fusions[1].stats[key]
        stats_diff[key] = str(stats_diff[key]) if stats_diff[key] < 0 else f"+{str(stats_diff[key])}"

    stats_print = {}
    for key in fusions[0].stats.keys():
            stats_print[key] = [fusions[0].stats[key],stats_diff[key],fusions[1].stats[key]]
    
    print("Typing and Stats")
    print(f"{'-'*20}")
    print(f"Head: {fusions[0].head.name.values[0]} | Body: {fusions[0].body.name.values[0]} | Typing: {list(fusions[0].types.values())}")
    sio = ['hp','atk','def','spatk','spdef','spe','total'] #stats in order
    aw = 7 #alignment width
    print(f"{sio[0]:^7}{sio[1]:^7}{sio[2]:^7}{sio[3]:^7}{sio[4]:^7}{sio[5]:^7}{sio[6]:^7}") #^ central alignment, < left, > right
    for i in range(3):
        print(f"{stats_print[sio[0]][i]:^7}{stats_print[sio[1]][i]:^7}{stats_print[sio[2]][i]:^7}{stats_print[sio[3]][i]:^7}{stats_print[sio[4]][i]:^7}{stats_print[sio[5]][i]:^7}{stats_print[sio[6]][i]:^7}")
    print(f"Head: {fusions[1].head.name.values[0]} | Body: {fusions[1].body.name.values[0]} | Typing: {list(fusions[1].types.values())}")
    print("")
    print(f"{'='*80}\nWeakness\n{'-'*20}")
    if set(fusions[1].types.values()) == set(fusions[0].types.values()):
        print("Fusions share the same typing")
        fusions[1].print_weakness()
    else:
        for fusion in fusions:
            print(f"Head: {fusion.head.name.values[0]} | Body: {fusion.body.name.values[0]} | Typing: {list(fusion.types.values())}")
            fusion.print_weakness()
            print(f"{'-'*20}")
    print("")
    print(f"{'='*80}\nAbilities\n{'-'*20}")
    fusion_abilities = fusions[0].possible_abilities()
    for key in fusion_abilities.keys():
        print(f"{key}: {fusion_abilities[key]}")
    

In [10]:
fusion_calculator("infernape","leafeon")

Typing and Stats
--------------------
Head: infernape | Body: leafeon | Typing: ['fire', 'grass']
  hp     atk    def   spatk  spdef   spe   total 
  72     108    110    89     69     99     547  
  +4     +2     +20    +15    +2     -4     +39  
  68     106    90     74     67     103    508  
Head: leafeon | Body: infernape | Typing: ['grass', 'fighting']

Weakness
--------------------
Head: infernape | Body: leafeon | Typing: ['fire', 'grass']
2.0 (3):['poison', 'flying', 'rock']
1.0 (11):['normal', 'fire', 'water', 'ice', 'fighting', 'ground', 'psychic', 'bug', 'ghost', 'dragon', 'dark']
0.5 (3):['electric', 'steel', 'fairy']
0.25 (1):['grass']
--------------------
Head: leafeon | Body: infernape | Typing: ['grass', 'fighting']
4.0 (1):['flying']
2.0 (5):['fire', 'ice', 'poison', 'psychic', 'fairy']
1.0 (6):['normal', 'fighting', 'bug', 'ghost', 'dragon', 'steel']
0.5 (6):['water', 'electric', 'grass', 'ground', 'rock', 'dark']
--------------------

Abilities
--------------------

In [11]:
fusion_calculator("venusaur","typhlosion")

Typing and Stats
--------------------
Head: venusaur | Body: typhlosion | Typing: ['grass', 'fire']
  hp     atk    def   spatk  spdef   spe   total 
  79     83     79     103    95     93     532  
  +1     +1     -2     -3     +5     +7     +9   
  78     82     81     106    90     86     523  
Head: typhlosion | Body: venusaur | Typing: ['fire', 'poison']

Weakness
--------------------
Head: venusaur | Body: typhlosion | Typing: ['grass', 'fire']
2.0 (3):['poison', 'flying', 'rock']
1.0 (11):['normal', 'fire', 'water', 'ice', 'fighting', 'ground', 'psychic', 'bug', 'ghost', 'dragon', 'dark']
0.5 (3):['electric', 'steel', 'fairy']
0.25 (1):['grass']
--------------------
Head: typhlosion | Body: venusaur | Typing: ['fire', 'poison']
4.0 (1):['ground']
2.0 (3):['water', 'psychic', 'rock']
1.0 (6):['normal', 'electric', 'flying', 'ghost', 'dragon', 'dark']
0.5 (5):['fire', 'ice', 'fighting', 'poison', 'steel']
0.25 (3):['grass', 'bug', 'fairy']
--------------------

Abilities
--------

In [12]:
fusion_calculator("mimikyu","tauros")

Typing and Stats
--------------------
Head: mimikyu | Body: tauros | Typing: ['ghost', 'normal']
  hp     atk    def   spatk  spdef   spe   total 
  61     96     90     46     93     105    491  
  -7     +3     +5     +3     +12    +5     +21  
  68     93     85     43     81     100    470  
Head: tauros | Body: mimikyu | Typing: ['normal', 'fairy']

Weakness
--------------------
Head: mimikyu | Body: tauros | Typing: ['ghost', 'normal']
2.0 (1):['dark']
1.0 (12):['fire', 'water', 'electric', 'grass', 'ice', 'ground', 'flying', 'psychic', 'rock', 'dragon', 'steel', 'fairy']
0.5 (2):['poison', 'bug']
0.0 (3):['normal', 'fighting', 'ghost']
--------------------
Head: tauros | Body: mimikyu | Typing: ['normal', 'fairy']
2.0 (2):['poison', 'steel']
1.0 (12):['normal', 'fire', 'water', 'electric', 'grass', 'ice', 'fighting', 'ground', 'flying', 'psychic', 'rock', 'fairy']
0.5 (2):['bug', 'dark']
0.0 (2):['ghost', 'dragon']
--------------------

Abilities
--------------------
regular: ['

In [13]:
fusion_calculator("azumarill","mamoswine")

Typing and Stats
--------------------
Head: azumarill | Body: mamoswine | Typing: ['water', 'ground']
  hp     atk    def   spatk  spdef   spe   total 
  103    103    80     63     73     70     492  
  -3     +27    +0     -3     +7     +10    +38  
  106    76     80     66     66     60     454  
Head: mamoswine | Body: azumarill | Typing: ['ice', 'fairy']

Weakness
--------------------
Head: azumarill | Body: mamoswine | Typing: ['water', 'ground']
4.0 (1):['grass']
1.0 (12):['normal', 'water', 'ice', 'fighting', 'ground', 'flying', 'psychic', 'bug', 'ghost', 'dragon', 'dark', 'fairy']
0.5 (4):['fire', 'poison', 'rock', 'steel']
0.0 (1):['electric']
--------------------
Head: mamoswine | Body: azumarill | Typing: ['ice', 'fairy']
4.0 (1):['steel']
2.0 (3):['fire', 'poison', 'rock']
1.0 (10):['normal', 'water', 'electric', 'grass', 'fighting', 'ground', 'flying', 'psychic', 'ghost', 'fairy']
0.5 (3):['ice', 'bug', 'dark']
0.0 (1):['dragon']
--------------------

Abilities
---------

In [14]:
fusion_calculator("hydreigon","volcarona")

Typing and Stats
--------------------
Head: hydreigon | Body: volcarona | Typing: ['dark', 'fire']
  hp     atk    def   spatk  spdef   spe   total 
  89     75     73     128    95     99     559  
  +2     -15    -8     -3     -5     +1     -28  
  87     90     81     131    100    98     587  
Head: volcarona | Body: hydreigon | Typing: ['bug', 'dragon']

Weakness
--------------------
Head: hydreigon | Body: volcarona | Typing: ['dark', 'fire']
2.0 (4):['water', 'fighting', 'ground', 'rock']
1.0 (7):['normal', 'electric', 'poison', 'flying', 'bug', 'dragon', 'fairy']
0.5 (6):['fire', 'grass', 'ice', 'ghost', 'dark', 'steel']
0.0 (1):['psychic']
--------------------
Head: volcarona | Body: hydreigon | Typing: ['bug', 'dragon']
2.0 (5):['ice', 'flying', 'rock', 'dragon', 'fairy']
1.0 (8):['normal', 'fire', 'poison', 'psychic', 'bug', 'ghost', 'dark', 'steel']
0.5 (4):['water', 'electric', 'fighting', 'ground']
0.25 (1):['grass']
--------------------

Abilities
--------------------
re

In [15]:
fusion_calculator("lucario","weavile")

Typing and Stats
--------------------
Head: lucario | Body: weavile | Typing: ['fighting', 'ice']
  hp     atk    def   spatk  spdef   spe   total 
  70     116    66     91     75     113    531  
  +0     +3     -2     +23    -5     +12    +31  
  70     113    68     68     80     101    500  
Head: weavile | Body: lucario | Typing: ['dark', 'steel']

Weakness
--------------------
Head: lucario | Body: weavile | Typing: ['fighting', 'ice']
2.0 (6):['fire', 'fighting', 'flying', 'psychic', 'steel', 'fairy']
1.0 (9):['normal', 'water', 'electric', 'grass', 'poison', 'ground', 'rock', 'ghost', 'dragon']
0.5 (3):['ice', 'bug', 'dark']
--------------------
Head: weavile | Body: lucario | Typing: ['dark', 'steel']
4.0 (1):['fighting']
2.0 (2):['fire', 'ground']
1.0 (4):['water', 'electric', 'bug', 'fairy']
0.5 (9):['normal', 'grass', 'ice', 'flying', 'rock', 'ghost', 'dragon', 'dark', 'steel']
0.0 (2):['poison', 'psychic']
--------------------

Abilities
--------------------
regular: ['st

In [16]:
fusion_calculator("jynx","garchomp")

Typing and Stats
--------------------
Head: jynx | Body: garchomp | Typing: ['ice', 'ground']
  hp     atk    def   spatk  spdef   spe   total 
  79     103    75     103    91     99     550  
  -14    +27    +20    +12    +3     +2     +50  
  93     76     55     91     88     97     500  
Head: garchomp | Body: jynx | Typing: ['dragon', 'psychic']

Weakness
--------------------
Head: jynx | Body: garchomp | Typing: ['ice', 'ground']
2.0 (5):['fire', 'water', 'grass', 'fighting', 'steel']
1.0 (11):['normal', 'ice', 'ground', 'flying', 'psychic', 'bug', 'rock', 'ghost', 'dragon', 'dark', 'fairy']
0.5 (1):['poison']
0.0 (1):['electric']
--------------------
Head: garchomp | Body: jynx | Typing: ['dragon', 'psychic']
2.0 (6):['ice', 'bug', 'ghost', 'dragon', 'dark', 'fairy']
1.0 (6):['normal', 'poison', 'ground', 'flying', 'rock', 'steel']
0.5 (6):['fire', 'water', 'electric', 'grass', 'fighting', 'psychic']
--------------------

Abilities
--------------------
regular: ['oblivious', 'f

In [17]:
fusion_calculator("infernape","mimikyu")

Typing and Stats
--------------------
Head: infernape | Body: mimikyu | Typing: ['fire', 'fairy']
  hp     atk    def   spatk  spdef   spe   total 
  69     94     77     86     82     100    508  
  +7     -5     +3     +18    -11    -4     +8   
  62     99     74     68     93     104    500  
Head: mimikyu | Body: infernape | Typing: ['ghost', 'fighting']

Weakness
--------------------
Head: infernape | Body: mimikyu | Typing: ['fire', 'fairy']
2.0 (4):['water', 'poison', 'ground', 'rock']
1.0 (6):['normal', 'electric', 'flying', 'psychic', 'ghost', 'steel']
0.5 (6):['fire', 'grass', 'ice', 'fighting', 'dark', 'fairy']
0.25 (1):['bug']
0.0 (1):['dragon']
--------------------
Head: mimikyu | Body: infernape | Typing: ['ghost', 'fighting']
2.0 (4):['flying', 'psychic', 'ghost', 'fairy']
1.0 (9):['fire', 'water', 'electric', 'grass', 'ice', 'ground', 'dragon', 'dark', 'steel']
0.5 (2):['poison', 'rock']
0.25 (1):['bug']
0.0 (2):['normal', 'fighting']
--------------------

Abilities
--

In [18]:
fusion_calculator("lycanroc","mimikyu",'midday form')

Typing and Stats
--------------------
Head: lycanroc | Body: mimikyu | Typing: ['rock', 'fairy']
  hp     atk    def   spatk  spdef   spe   total 
  68     98     75     53     78     101    473  
  +7     -8     +5     +2     -13    -5     -12  
  61     106    70     51     91     106    485  
Head: mimikyu | Body: lycanroc | Typing: ['ghost', 'rock']

Weakness
--------------------
Head: lycanroc | Body: mimikyu | Typing: ['rock', 'fairy']
4.0 (1):['steel']
2.0 (3):['water', 'grass', 'ground']
1.0 (8):['electric', 'ice', 'fighting', 'poison', 'psychic', 'rock', 'ghost', 'fairy']
0.5 (5):['normal', 'fire', 'flying', 'bug', 'dark']
0.0 (1):['dragon']
--------------------
Head: mimikyu | Body: lycanroc | Typing: ['ghost', 'rock']
2.0 (6):['water', 'grass', 'ground', 'ghost', 'dark', 'steel']
1.0 (6):['electric', 'ice', 'psychic', 'rock', 'dragon', 'fairy']
0.5 (3):['fire', 'flying', 'bug']
0.25 (1):['poison']
0.0 (2):['normal', 'fighting']
--------------------

Abilities
---------------